# Hotel booking baseline
An attempt to solve the problem through function calling without fine-tuning

In [39]:
import json
import logging
import requests
from typing import Callable, Any
from datetime import date

from ollama import ChatResponse, chat
from pydantic import BaseModel, Field

from data_models import HotelData, AvailabilityRequest, AvailabilityResponse, BookingRequest, BookingResponse

In [40]:
logging.basicConfig(level=logging.DEBUG)    

In [41]:
def get_hotels(city: str, checkin_date: date, checkout_date: date):
    """
    Get data on all hotels available in a given city on given dates.
    
    Args:
        city: City where available hotels should be found
        checkin_date: Date when the user wants to check in (format: date from the datetime package)
        checkout_date: Date when the user wants to check out (format: date from the datetime package)
    
    Returns:
        JSON with the following keys:
            success: a Boolean with value True if there are hotels with rooms available for booking in the given city on the given dates and False if no hotel rooms are available for booking
            error_message: A string giving reasons, if any, for not fulfilling the request, e.g. city not found or no free rooms on the given dates or None if the search was successfull
            available_hotels: a list where each element is a dictionary with the following keys:
                name: hotel name
                star_rating: star rating of the hotel (an integer between 1 and 5)
                price: room price per night
        
    """
    api_url = 'http://127.0.0.1:8000/get_hotels'
    headers = {"Content-Type": "application/json"}
    response = requests.post(api_url,
                         headers=headers,
                         json={'city': city, 'checkin_date': checkin_date, 'checkout_date': checkout_date})
    logging.debug(f'Response from the get_hotel endpoint: {response}')
    return response.json()

In [42]:
def book_hotel(name: str, city: str, checkin_date: date, checkout_date: date):
    """
    Book a hotel room as per the user request.
    
    Args:
        name: name of the hotel where a room should be booked
        city: city the hotel is located
        checkin_date: date when the user wants to check in
        checkout_date: date when the user wants to check out
    
    Returns:
        JSON with the following keys:
            success: a Boolean with value True if booking was successful and False if not
            message: string describing the outcome, e.g. confirming the booking or giving reasons for failure
    """
    api_url = 'http://127.0.0.1:8000/book'
    headers = {"Content-Type": "application/json"}
    response = requests.post(api_url,
                         headers=headers,
                         json={'city': 'Moscow', 'name': name, 'checkin_date': checkin_date, 'checkout_date': checkout_date})
    return response.json()

In [43]:
available_functions = {
    "get_hotels": get_hotels,
    "book_hotel": book_hotel
}

In [44]:
class FunctionCall(BaseModel):
    """Single function call to be returned by an LLM"""
    name: str = Field(..., description='Name of the function to be called')
    arguments: dict[str, str | None] = Field(..., description='Arguments of the function to be called')

In [45]:
class FunctionCalls(BaseModel):
    """All function calls to be returned by an LLM to process a user's request"""
    function_calls: list[FunctionCall] = Field(default=[], description='List of function calls returned by an LLM')
    comment: str | None = Field(..., description='LLM comments on the data provided, for example pointing out to missing arguments')

In [46]:
class BookingAgent:
    def __init__(self, functions: dict[str, Callable], model_name: str, system_message_1: str, system_message_2: str):
        self.functions = functions
        self.model_name = model_name
        self.system_message_1 = system_message_1
        self.system_message_2 = system_message_2

    def __call__(self, user_request: str):
        """Respond to user queries"""

        # First call to an LLM
        response: ChatResponse = chat(
            self.model_name,
            messages=[
                {'role': 'system', 'content': self.system_message_1},
                {'role': 'user', 'content': user_request}
            ],
            tools=list(self.functions.values()),
            stream=False,
            format=FunctionCalls.model_json_schema()
        )
        logging.info(f'Response from the first call to LLM: {response}')

        # Function calls
        llm_response = json.loads(response.message.content)
        logging.debug(f'Parsed llm response: {llm_response}')
        api_response = []
        assistant_message = None
        if llm_response.get('function_calls'):
            try:
                for function_call in llm_response['function_calls']:
                    func_name = function_call['name']
                    if func_name in self.functions:
                        api_response.extend(
                            [
                                f'Response from function {func_name}:',
                                str(self.functions[func_name](**function_call['arguments']))
                            ]
                        )
            except Exception as e:
                logging.error(f'Function calls failed with error: {str(e)}')
                api_response = [llm_response.get('comment')]
        else:
            api_response = [llm_response.get('comment')]
        logging.info(f'Response from the booking service API: {api_response}')
        
        # Second call to the LLM
        prompt = f'''
User request:
{user_request}
Response from the booking service API:
{'/n'.join(api_response)}
        '''
        logging.debug(f'Prompt:\n{prompt}')
        response: ChatResponse = chat(
            self.model_name,
            messages=[
                {'role': 'system', 'content': self.system_message_2},
                {'role': 'user', 'content': prompt}
            ],
            stream=False
        )
        logging.info(f'Response from the second call to LLM: {response}')
        return response.message.content

In [47]:
system_message_1 = '''You are a booking agent helping users to find available hotels or to book a room in a specific hotel. You accomplish this by calling relevant functions.

**Crucially, you MUST ONLY use data provided DIRECTLY by the user.
DO NOT invent any data or make any assumptions. 
If the user does not provide all the necessary information, return a JSON with an empty list of function calls and an explanation of the missing data.**
 
If the user wants to book a room in a specific hotel, do not search for other options, just call the function for booking the room.

**Your response MUST be valid JSON conforming to the following structure:**
{
    "function_calls": [
        {
            "name":"function name",
            "arguments":{
                "arg1": "value1",
                "arg2": "value2"
            }
        }
    ],
    "comment": "Explanation of missing data or success."
}

If the user request does not include all required parameters for the function call, return JSON with the above format containing an empty list of function calls and a comment explaining which data are missing.

Example #1:
User query: "Book me a room in The Luxury Collection hotel, Guangzhou, from August 1, 2025 to August 7, 2025".
Response: {
    "function_calls": [{"name":"book_hotel", "arguments":{"city":"Guangzhou", name: "The Luxury Collection", "checkin_date":"2025-08-01", "checkout_date":"2025-08-07"}}],
    "comment": ""
}

Example #2:
User query: "Find me hotels in Tokio".
Response: {
    "function_calls": [],
    "comment": "Missing checkin and checkout date"
}

Example #3:
User query: "Which hotels are available in Seoul between August 1, 2025 and August 2, 2025".
Response: {
    "function_calls": [{"name":"get_hotels", "arguments":{"city":"Seoul", "checkin_date":"2025-08-01", "checkout_date":"2025-08-02"}}],
    "comment": ""
}


Example #4:
User query: "Find me a hotel in London after September 1, 2026".
Response: {
    "function_calls": [],
    "comment": "Missing checkout date"
}


'''
system_message_2 = '''You are a booking agent helping users to find available hotels or to book a room in a hotel.
The user request was forwarded to the booking service API and it returned a response.

**Crucially, you MUST ONLY use data provided IN THE API RESPONSE.  DO NOT invent any data or make any assumptions. If there is no response, inform the user that the service is not available.**

Answer questions and requests disrelated to your tasks by decribing the tasks you are suppposed to accomplish.
'''

In [48]:
booking_agent = BookingAgent(
    functions=available_functions,
    model_name='mistral',
    system_message_1=system_message_1,
    system_message_2=system_message_2
)

Valid request to find hotels:

In [49]:
result = booking_agent(user_request="Which hotels are available in Moscow between 20 August 2025 and 21 August 2025?")
print(result)

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x79ebba6016e0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Sun, 20 Jul 2025 09:24:00 GMT'), (b'Content-Length', b'467')])
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:rece

 Based on the response from the booking service API, there are several hotels available in Moscow between August 20th and 21st, 2025. Here's a list of the hotels along with their star ratings and prices:

1. CityView Boutique - Star Rating: 4, Price: $400 per night
2. Masterpiece Manor - Star Rating: 4, Price: $400 per night
3. Supreme Solitude - Star Rating: 4, Price: $400 per night
4. Whispering Dreams - Star Rating: 1, Price: $100 per night

Please note that the prices are in USD and they might not reflect the actual cost due to possible currency conversions and taxes that may apply during your booking process. I would recommend visiting the hotel's official website or contacting them directly for more detailed information about their rates and services.


Invalid request to find hotels (missing dates):

In [50]:
result = booking_agent(user_request="Which hotels are available in Moscow?")
print(result)

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Sun, 20 Jul 2025 09:24:07 GMT'), (b'Content-Length', b'495')])
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
INFO:root:Response from the first call to LLM: model='mistral' created_at='2025-07-20T09:24:07.4416054Z' done=True done_reason='s

 I'm sorry for the inconvenience, but it seems there's an issue with the provided check-in and checkout dates. The booking service API requires both a check-in date and a checkout date to provide hotel options. Could you please provide valid dates for your stay in Moscow? Once we have those, I'll be able to help you find available hotels.


Valid request to book a room:

In [51]:
result = booking_agent(user_request="Book me a room in Masterpiece Manor, Moscow, from July 30, 2025 to August 30, 2025")
print(result)

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Sun, 20 Jul 2025 09:24:12 GMT'), (b'Content-Length', b'512')])
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
INFO:root:Response from the first call to LLM: model='mistral' created_at='2025-07-20T09:24:12.170383568Z' done=True done_reason=

 Based on the response from the booking service API, I can confirm that your request for a room at Masterpiece Manor in Moscow from July 30, 2025, to August 30, 2025, has been successfully booked. The hotel has confirmed your reservation.

Here are the details of your booking:
- Hotel name: Masterpiece Manor
- City: Moscow
- Check-in date: July 30, 2025
- Check-out date: August 30, 2025
- Booking status: Confirmed


Valid request, but no hotels are available:

In [52]:
result = booking_agent(user_request="Which hotels are available in Cape Town between 20 August 2025 and 21 August 2025?")
print(result)

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Sun, 20 Jul 2025 09:24:17 GMT'), (b'Content-Length', b'470')])
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
INFO:root:Response from the first call to LLM: model='mistral' created_at='2025-07-20T09:24:17.744358284Z' done=True done_reason=

 I'm sorry for any inconvenience, but it appears that there are no hotels available in Cape Town between August 20, 2025, and August 21, 2025. The booking service API did not return any hotels that fit these criteria. If you have different dates or would like to check for other locations, I'd be more than happy to assist you further.


Request disrelated to booking:

In [53]:
result = booking_agent(user_request="How is your Mom?")
print(result)

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Sun, 20 Jul 2025 09:24:22 GMT'), (b'Content-Length', b'563')])
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
INFO:root:Response from the first call to LLM: model='mistral' created_at='2025-07-20T09:24:22.356377714Z' done=True done_reason=

 I'm sorry for any inconvenience. As a booking agent, my primary function is to assist you in finding available hotels or booking a room in a hotel based on your requirements. The service has just responded with an error related to the check-out date format. Let's focus on finding hotels instead. Could you please provide me with the check-out date in a valid format, such as YYYY-MM-DD? This will help us proceed with the booking process smoothly.
